In [ ]:
#import libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
#load the dataset
data = pd.read_csv(r'Thesis data\data_app_cat-241.csv', dtype='unicode')

In [ ]:
#create month column
data['month'] = pd.to_datetime(data['startTime']).dt.month

Data cleaning 

In [ ]:
#group the data by studentID, session, month

sessions = data.groupby(['StudentID'])['session', 'month'].nunique().reset_index()

In [ ]:
#remove participants with less than 200 sessions and less than 3 months participation in the study

cleaned = sessions[(sessions['session'] > 200)]

cleaned = cleaned[(cleaned['month']>=3)]

In [ ]:
#filter only the students who participated more than 3 months and have more than 200 sessions

new_data = data[data['StudentID'].isin(cleaned['StudentID'])]



Exploratory Data Analysis

In [ ]:
student_session =new_data.groupby(['StudentID'])['session'].nunique().reset_index()

In [ ]:
#create a density plot to see the sessions distribution


sns.kdeplot(cleaned['session'], shade=True)

#add labels and title
plt.xlabel('X-axis')
plt.ylabel('Density')
plt.title('Density Plot')

plt.xlim(0, max(cleaned['session']))


plt.show()

In [ ]:
#create a barplot to see the most active months of phone usage

sns.histplot(cleaned['month'])

#add labels and title
plt.xlabel('X-axis')
plt.ylabel('Density')
plt.title('Density Plot')

plt.xlim(min(cleaned['month']), max(cleaned['month']))

plt.show()

In [ ]:
#Plot of the monimum, maximum and average session 

min_session = student_session['session'].min()
max_session = student_session['session'].max()
average_session = student_session['session'].mean()

#create a DataFrame for the statistics
statistics_df = pd.DataFrame({'Statistics': ['Minimum', 'Maximum', 'Average'],
                              'Session': [min_session, max_session, average_session]})

#display the table
statistics_df

In [ ]:
student_month =new_data.groupby(['StudentID'])['month', 'session'].nunique().reset_index()

In [ ]:
#percentage of students who participated for 3-4 months and students who participated for 5-6 months

import matplotlib.pyplot as plt

#calculate the number of people who participated for 3 and 4 months
three_four_months_count = len(student_month[(student_month['month'] <= 3) & (student_month['month'] <= 4)])

#calculate the number of people who participated for 5 and 6 months
five_six_months_count = len(student_month[(student_month['month'] >= 5) & (student_month['month'] <= 6)])

#calculate the total number of participants
total_participants = len(student_month)

#calculate the percentage of people who participated for 3 and 4 months
three_four_months_percentage = (three_four_months_count / total_participants) * 100

#calculate the percentage of people who participated for 5 and 6 months
five_six_months_percentage = (five_six_months_count / total_participants) * 100

#create a bar chart to visualize the percentages
plt.figure(figsize=(8, 6))
plt.bar(['3-4 Months', '5-6 Months'], [three_four_months_percentage, five_six_months_percentage], color=['blue', 'green'])

#add labels and title
plt.xlabel('Participation Duration')
plt.ylabel('Percentage')
plt.title('Percentage of Participants by Duration')

# show the percentages above the bars
for i, v in enumerate([three_four_months_percentage, five_six_months_percentage]):
    plt.text(i, v + 1, f'{v:.2f}%', ha='center', color='black', fontweight='bold')

plt.show()

In [ ]:
#plot the most used applications

import matplotlib.pyplot as plt

# Calculate the sum of minutes for each category
category_minutes = new_data.groupby('better_category')['minutes'].sum()

# Get the top 10 categories with the highest minutes
top_10_categories = category_minutes.nlargest(10)

# Plot the bar chart for the top 10 categories
plt.figure(figsize=(10, 6))
sns.barplot(x=top_10_categories.index, y=top_10_categories.values, palette='viridis')

# Add a title and labels
plt.title('Top 10 Categories with Highest Amount of Minutes')
plt.xlabel('Categories')
plt.ylabel('Total Minutes among students')

# Rotate x-axis tick labels for better readability
plt.xticks(rotation=90)

# Show the plot
plt.tight_layout()
plt.show()


Feature Engineering

1. Ratio of phone usage during 6 hour daily windows (6am-12pm, 12pm-6pm, 6pm-12am, 12pm-6am)

In [ ]:
#create daily windows; measure phone usage in the windows
# ratio of phone usage in window

new_data['dailyWindow'] = pd.cut(pd.to_datetime(new_data['startTime']).dt.hour,
                             bins=[0, 6, 12, 18, 24],
                             labels=['0-6', '6-12', '12-18', '18-24'],
                             include_lowest=True)

In [ ]:
#group the data on students, month and window
#aggregate a list with the minutes to prevent imputiong 0 for the months students' didn't participate
usage_per_window = new_data.groupby(['StudentID', 'month', 'dailyWindow'])['minute'].agg(list).reset_index()

In [ ]:
#sum the minutes in the lists
usage_per_window['minute'] = usage_per_window['minute'].apply(np.sum)

In [ ]:
#rename minutes column
usage_per_window.rename(columns = {'minute': 'total_time_per_window'}, inplace=True)

In [ ]:
#calculate the ratio per window

usage_per_window_ratio = new_data.groupby(['StudentID', 'month', 'dailyWindow'])['minute'].agg(list).reset_index()

usage_per_window_ratio['minute'] = usage_per_window_ratio['minute'].apply(np.sum)



# Calculate the total phone usage duration per student
total_usage_per_student_month = new_data.groupby(['StudentID', 'month'])['minute'].agg(list).reset_index(name='total')

total_usage_per_student_month['total'] = total_usage_per_student_month['total'].apply(np.sum)

In [ ]:
#merge usage_per_window_ratio table and total_usage_per_student_month
usage_per_window_ratio.set_index('StudentID', inplace=True)
usage_per_window_ratio['StudentID'] = usage_per_window_ratio['StudentID'].astype(int)
merged_df_ratio = pd.merge(usage_per_window_ratio, total_usage_per_student_month, on=['StudentID', 'month'])

#create ratio column
merged_df_ratio['ratio'] = merged_df_ratio['minute'] / merged_df_ratio['total']

In [ ]:
#rename to ratio_total_phone_usage

merged_df_ratio.rename(columns={'ratio': 'ratio_total_usage'}, inplace=True)

2. Ratio of app usage during 6 hour daily windows (6am-12pm, 12pm-6pm, 6pm-12am, 12pm-6am)

2.1. Ratio Social Networking

In [ ]:
# Group the data by ID, month, and time window
grouped = data.groupby(['StudentID', pd.Grouper(key='startTime', freq='6H'), pd.Grouper(key='endTime', freq='M'), 'timeWindow'])

# Calculate the total time spent on gaming per ID, month, and time window
result = grouped.apply(lambda x: x.loc[x['better_category'] == 'Social_Networking', 'timeMinutes'].sum())

result = result.reset_index(name='total_time_SN')

# Rename the columns to match the desired output
result.columns = ['StudentID', 'startTime', 'endTime', 'timeWindow', 'total_time_SN']

In [ ]:
result['endTime'] = pd.to_datetime(result['endTime'])

#group by student id and aggregate time monthly and aggregate a list with total minutes on media per time window

group = result.groupby(['StudentID', pd.Grouper(key='endTime', freq='M'), 'timeWindow'])['total_time_SN'].agg(list)

In [ ]:
#create a df
df = pd.DataFrame(group)
df.reset_index(inplace=True)

In [ ]:
# sum the total minutes on social media for the whole month
group2 = result.groupby(['StudentID', 'endTime'])['total_time_SN'].agg(sum)

#merge the time windows with the total time on media
merged_df = pd.merge(df, group2, on=['StudentID', 'endTime'])



In [ ]:
#calculate the ratio on social networking per time window
merged_df['ratio_SN'] = merged_df['total_time_SN_time_window'] / merged_df['total_time_SN']

In [ ]:
final['ratio_SN'] = merged_df['ratio_SN']

Ratio Instant Messaging

In [ ]:
# Group the data by ID, month, and time window
grouped = data.groupby(['StudentID', pd.Grouper(key='startTime', freq='6H'), pd.Grouper(key='endTime', freq='M'), 'timeWindow'])

# Calculate the total time spent on gaming per ID, month, and time window
result = grouped.apply(lambda x: x.loc[x['better_category'] == 'Instant_Messaging', 'timeMinutes'].sum())

result = result.reset_index(name='total_time_IM')

# Rename the columns to match the desired output
result.columns = ['StudentID', 'startTime', 'endTime', 'timeWindow', 'total_time_IM']


In [ ]:
result['endTime'] = pd.to_datetime(result['endTime'])

#group by student id and aggregate time monthly and aggregate a list with total minutes on media per time window
group_im = result.groupby(['StudentID', pd.Grouper(key='endTime', freq='M'), 'timeWindow'])['total_time_IM'].agg(list)

In [ ]:
# sum the total minutes on  media for the whole month
group2_im = result.groupby(['StudentID', 'endTime'])['total_time_IM'].agg(sum)

In [ ]:
#merge the time windows with the total time on media
merged_df_im = pd.merge(group_im, group2_im, on=['StudentID', 'endTime'])

In [ ]:
#calculate ratio per window
merged_df_im['ratio_IM'] = merged_df_im['total_time_IM_time_window'] / merged_df_im['total_time_IM']

In [ ]:
final['ratio_IM'] = merged_df_im['ratio_IM'] 

Ratio YouTube

In [ ]:
# Group the data by ID, month, and time window
grouped = data.groupby(['StudentID', pd.Grouper(key='startTime', freq='6H'), pd.Grouper(key='endTime', freq='M'), 'timeWindow'])

# Calculate the total time spent on gaming per ID, month, and time window
result = grouped.apply(lambda x: x.loc[x['better_category_hybrid'] == 'YouTube', 'timeMinutes'].sum())

result = result.reset_index(name='total_time_YT')

# Rename the columns to match the desired output
result.columns = ['StudentID', 'startTime', 'endTime', 'timeWindow', 'total_time_YT']

In [ ]:
result['endTime'] = pd.to_datetime(result['endTime'])

#group by student id and aggregate time monthly and aggregate a list with total minutes on media per time window
group_yt = result.groupby(['StudentID', pd.Grouper(key='endTime', freq='M'), 'timeWindow'])['total_time_YT'].agg(list)

In [ ]:
# sum the total minutes on  media for the whole month
group2_yt = result.groupby(['StudentID', 'endTime'])['total_time_YT'].agg(sum)

#merge the time windows with the total time on media
merged_df_yt = pd.merge(group_yt, group2_yt, on=['StudentID', 'endTime'])


In [ ]:
#calculate youtube ratio per daily window 
merged_df_yt['ratio_YT'] = merged_df_yt['total_time_YT_x'] / merged_df_yt['total_time_YT_y']

In [ ]:
final['ratio_YT'] = merged_df_yt['ratio_YT'] 

Ratio Streaming Services

In [ ]:
# Group the data by ID, month, and time window
grouped = data.groupby(['StudentID', pd.Grouper(key='startTime', freq='6H'), pd.Grouper(key='endTime', freq='M'), 'timeWindow'])

# Calculate the total time spent on gaming per ID, month, and time window
result = grouped.apply(lambda x: x.loc[x['better_category'] == 'Streaming_Services', 'timeMinutes'].sum())

result = result.reset_index(name='total_time_SS')

# Rename the columns to match the desired output
result.columns = ['StudentID', 'startTime', 'endTime', 'timeWindow', 'total_time_SS']

result['endTime'] = pd.to_datetime(result['endTime'])

##group by student id and aggregate time monthly and aggregate a list with total minutes on media per time window
group_ss = result.groupby(['StudentID', pd.Grouper(key='endTime', freq='M'), 'timeWindow'])['total_time_SS'].agg(list)

In [ ]:
# sum the total minutes on  media for the whole month
group2_ss = result.groupby(['StudentID', 'endTime'])['total_time_SS'].agg(sum)

#merge the time windows with the total time on media
merged_df_ss = pd.merge(group_ss, group2_ss, on=['StudentID', 'endTime'])

#calculate Ratio
merged_df_ss['ratio_SS'] = merged_df_ss['total_time_SS_x'] / merged_df_ss['total_time_SS_y']


In [ ]:
final['ratio_SS'] = merged_df_ss['ratio_SS'] 

Ratio Internet Browsing

In [ ]:
# Group the data by ID, month, and time window
grouped = data.groupby(['StudentID', pd.Grouper(key='startTime', freq='6H'), pd.Grouper(key='endTime', freq='M'), 'timeWindow'])

# Calculate the total time spent on gaming per ID, month, and time window
result = grouped.apply(lambda x: x.loc[x['better_category'] == 'Internet Browsing', 'timeMinutes'].sum())

result = result.reset_index(name='total_time_BR')

# Rename the columns to match the desired output
result.columns = ['StudentID', 'startTime', 'endTime', 'timeWindow', 'total_time_BR']

result['endTime'] = pd.to_datetime(result['endTime'])

##group by student id and aggregate time monthly and aggregate a list with total minutes on media per time window
group_br = result.groupby(['StudentID', pd.Grouper(key='endTime', freq='M'), 'timeWindow'])['total_time_BR'].agg(list)

In [ ]:
# sum the total minutes on  media for the whole month
group2_br = result.groupby(['StudentID', 'endTime'])['total_time_BR'].agg(sum)

#merge the time windows with the total time on media
merged_df_br = pd.merge(group_br, group2_br, on=['StudentID', 'endTime'])

#calculate Ratio
merged_df_br['ratio_BR'] = merged_df_br['total_time_BR_x'] / merged_df_br['total_time_BR_y']

In [ ]:
final['ratio_BR'] = merged_df_br['ratio_BR'] 

Ratio Dialer

In [ ]:
# Group the data by ID, month, and time window
grouped = data.groupby(['StudentID', pd.Grouper(key='startTime', freq='6H'), pd.Grouper(key='endTime', freq='M'), 'timeWindow'])

# Calculate the total time spent on gaming per ID, month, and time window
result = grouped.apply(lambda x: x.loc[x['better_category'] == 'Dialer', 'timeMinutes'].sum())

result = result.reset_index(name='total_time_dlr')

# Rename the columns to match the desired output
result.columns = ['StudentID', 'startTime', 'endTime', 'timeWindow', 'total_time_dlr']

result['endTime'] = pd.to_datetime(result['endTime'])

##group by student id and aggregate time monthly and aggregate a list with total minutes on media per time window
group_dlr = result.groupby(['StudentID', pd.Grouper(key='endTime', freq='M'), 'timeWindow'])['total_time_dlr'].agg(list)

In [ ]:
# sum the total minutes on  media for the whole month
group2_dlr = result.groupby(['StudentID', 'endTime'])['total_time_dlr'].agg(sum)

#merge the time windows with the total time on media
merged_df_dlr = pd.merge(group_dlr, group2_dlr, on=['StudentID', 'endTime'])

#calculate Ratio
merged_df_dlr['ratio_dlr'] = merged_df_dlr['total_time_dlr_x'] / merged_df_dlr['total_time_dlr_y']

In [ ]:
final['ratio_dlr'] = merged_df_dlr['ratio_dlr']

Ratio Email

In [ ]:
# Group the data by ID, month, and time window
grouped = data.groupby(['StudentID', pd.Grouper(key='startTime', freq='6H'), pd.Grouper(key='endTime', freq='M'), 'timeWindow'])

# Calculate the total time spent on gaming per ID, month, and time window
result = grouped.apply(lambda x: x.loc[x['better_category'] == 'Email', 'timeMinutes'].sum())

result = result.reset_index(name='total_time_eml')

# Rename the columns to match the desired output
result.columns = ['StudentID', 'startTime', 'endTime', 'timeWindow', 'total_time_eml']

result['endTime'] = pd.to_datetime(result['endTime'])

##group by student id and aggregate time monthly and aggregate a list with total minutes on media per time window
group_eml = result.groupby(['StudentID', pd.Grouper(key='endTime', freq='M'), 'timeWindow'])['total_time_eml'].agg(list)

In [ ]:
# sum the total minutes on  media for the whole month
group2_eml = result.groupby(['StudentID', 'endTime'])['total_time_eml'].agg(sum)

#merge the time windows with the total time on media
merged_df_eml = pd.merge(group_eml, group2_eml, on=['StudentID', 'endTime'])

#calculate Ratio
merged_df_eml['ratio_eml'] = merged_df_eml['total_time_eml_x'] / merged_df_eml['total_time_eml_y']

In [ ]:
final['ratio_eml'] = merged_df_eml['ratio_eml']

In [ ]:
#pivot table
m_w_order = ['January_24-6', 'January_6-12', 'January_12-18', 'January_18-24', 'Fabruary_24-6', 'February_6-12', 'February_12-18', 'Fabruary_18-24', 'March_24-6', 'March_6-12', 'March_12-18', 'March_18-24', 'April_24-6', 'April_6-12', 'April_12-18', 'April_18-24', 'May_24-6', 'May_6-12', 'May_12-18', 'May_18-24', 'June_24-6', 'June_6-12', 'June_12-18', 'June_18-24', 'July', 'August', 'September', 'October', 'November', 'December']

windows_new = pd.pivot_table(final, values=final.columns, index=final['StudentID'], columns=['month_window'])

windows_new = windows_new.reindex(m_w_order, axis=1, level=1)

In [ ]:
#flatten
new_columns = [f"{top_col}_{sub_col}" for top_col, sub_col in windows_new.columns]

windows_new.columns = new_columns
windows_new.reset_index(inplace=True)

In [ ]:

windows_new.to_csv(r'new_removed_tables\ratio_daily_window_apps', index=False)

Trend Slopes

1. Trend slopes of phone usage for all of the months

In [ ]:
#group the data
gr_slope = new_data.groupby(['StudentID', 'month', 'day'])['minute'].sum().reset_index()

In [ ]:
from scipy.stats import linregress
gr_slope['minute'] = pd.to_numeric(gr_slope['minute'])


#group the data by 'UserID' in each sample
grouped_data = gr_slope.groupby('StudentID')


#create empty dataframes to store the results for each sample
result_df = pd.DataFrame(columns=['StudentID', 'time_slope_on_phone'])


#iterate over each group in sample 1
for student_id, group in grouped_data:
    # Extract the x values for linear regression
    x = np.arange(len(group))  # Assuming time is represented by the index of the dataframe
    
    #perform linear regression for each variable and calculate the trend slope
    s1 = linregress(x, group['minute']).slope

    
    # append the result to the result dataframe for sample 1
    result_df = result_df.append({'StudentID': student_id,'time_slope_on_phone': s1}, ignore_index=True)

2. Trens slopes of phone usage for months: January-March and April-June

In [ ]:
#create the first sample from the beginning month to the end of the third month
sample1 = gr_slope[gr_slope['month'].isin([1, 2, 3])]

#create the second sample from the beginning of the fourth month to the end of the sixth month
sample2 = gr_slope[gr_slope['month'].isin([4, 5, 6])]

#group the data by 'UserID' in each sample
grouped_data1 = sample1.groupby('StudentID')
grouped_data2 = sample2.groupby('StudentID')

#create empty dataframes to store the results for each sample
result_df1 = pd.DataFrame(columns=['StudentID', 'time_slope_march'])
result_df2 = pd.DataFrame(columns=['StudentID', 'time_slope_june'])

#iterate over each group in sample 1
for student_id, group in grouped_data1:
    #extract the x values for linear regression
    x = np.arange(len(group))  # Assuming time is represented by the index of the dataframe
    
    #perform linear regression for each variable and calculate the trend slope
    s1 = linregress(x, group['minute']).slope

    
    # append the result to the result dataframe for sample 1
    result_df1 = result_df1.append({'StudentID': student_id,'time_slope_march': s1}, ignore_index=True)

#iterate over each group in sample 2
for student_id, group in grouped_data2:
    #extract the x values for linear regression
    x = np.arange(len(group))  # Assuming time is represented by the index of the dataframe
    
    #perform linear regression for each variable and calculate the trend slope
    s2 = linregress(x, group['minute']).slope


    result_df2 = result_df2.append({'StudentID': student_id,
                                    'time_slope_june': s2}, ignore_index=True)

In [ ]:
#save results

result_df1.to_csv(r'new_removed_tables\phone_usage_trend_slope_march.csv', index=False)
result_df2.to_csv(r'new_removed_tables\phone_usage_trend_slope_june.csv', index=False)
result_df = pd.read_csv(r'new_removed_tables\phone_usage_trend_slope_all_months.csv')




3. Trend slopes of app usage for all of the months
- app categories: 'Instant_Messaging', 'Social_Networking', 'Streaming_Services', 'Email', 'Dialer', 'Internet_Browser'

In [ ]:
#create list of the values

trend_columns = ['Instant_Messaging', 'Social_Networking', 'Streaming_Services', 'Email', 'Dialer', 'Internet_Browser']



#group the data by 'StudentID' in each sample
grouped_data = new_data.groupby('StudentID')


# create empty dataframes to store the results for each sample
result_df = pd.DataFrame(columns=['StudentID'] + trend_columns)


# calculate trend slopes for sample 1
for student_id, group in grouped_data:
    trends1 = {'StudentID': student_id}
    for category in trend_columns:
        category_data = group[group['better_category'] == category]
        if len(category_data) > 1:
            x = np.arange(len(category_data))
            y = category_data['minute']
            slope = linregress(x, y).slope
        else:
            slope = np.nan
        trends1[category] = slope
    result_df = result_df.append(trends1, ignore_index=True)


In [ ]:
#save results
result_df.to_csv(r'new_removed_tables\app_usage_all_months.csv')

4. Trend slopes of app usage for months: January-March and April-June

In [ ]:
from scipy.stats import linregress

#create list of the values
trend_columns = ['Instant_Messaging', 'Social_Networking', 'Streaming_Services', 'Email', 'Dialer', 'Internet_Browser']

# Group the data by 'StudentID' in each sample
grouped_data = new_data.groupby('StudentID')

# Create empty dataframes to store the results for each sample
result_df_march = pd.DataFrame(columns=['StudentID'] + [col + '_march' for col in trend_columns])
result_df_june = pd.DataFrame(columns=['StudentID'] + [col + '_june' for col in trend_columns])

# Calculate trend slopes for sample 1 (March)
for student_id, group in grouped_data:
    trends_march = {'StudentID': student_id}
    march_data = group[group['month'].between(1, 3)]
    for category in trend_columns:
        category_data = march_data[march_data['better_category'] == category]
        if len(category_data) > 1:
            x = np.arange(len(category_data))
            y = category_data['minute']
            slope = linregress(x, y).slope
        else:
            slope = np.nan
        trends_march[category + '_march'] = slope
    result_df_march = result_df_march.append(trends_march, ignore_index=True)

# Calculate trend slopes for sample 2 (June)
for student_id, group in grouped_data:
    trends_june = {'StudentID': student_id}
    june_data = group[group['month'].between(4, 6)]
    for category in trend_columns:
        category_data = june_data[june_data['better_category'] == category]
        if len(category_data) > 1:
            x = np.arange(len(category_data))
            y = category_data['minute']
            slope = linregress(x, y).slope
        else:
            slope = np.nan
        trends_june[category + '_june'] = slope
    result_df_june = result_df_june.append(trends_june, ignore_index=True)

In [ ]:
#save results
result_df_march.to_csv(r'new_removed_tables\app_usage_march.csv')
result_df_june.to_csv(r'new_removed_tables\app_usage_june.csv')

In [ ]:
#concat the slopes tables

result_df.set_index('StudentID', inplace=True)
result_df_march.set_index('StudentID', inplace=True)
result_df_june.set_index('StudentID', inplace=True)

merged_tables3 = result_df.merge(result_df_march, on='StudentID', how='left')
merged_tables4 =  merged_tables3.merge(result_df_june, on='StudentID', how='left')


In [ ]:
#save concatenated columns
merged_tables4.to_csv(r'new_removed_tables\app_usage_slopes_all_march_june_ANALYSIS.csv')